In [ ]:
#cantidad d documentos
cant=200

# 1) Cargar los archivos en formato de texto y guardarlos en coleccion mongodb


In [ ]:
import os
import pymongo
import datetime
from pymongo import MongoClient
import re
from bs4 import BeautifulSoup
import os
import re
from time import time 
from ipywidgets import widgets
from IPython.core.display import display, HTML
from IPython.display import display
from itertools import islice
matriz = []
tiempo_inicial = time() 
formato=".txt"
stopwords= "(\n|\t|\-|\+|\/|'s | a | about | above | after | again | against | all | am | an | and | any | are | aren't | as | at | be | because | been | before | being | below | between | both | but | by | can't | cannot | could | couldn't | did | didn't | do | does | doesn't | doing | don't | down | during | each | few | for | from | further | had | hadn't | has | hasn't | have | haven't | having | he | he'd | he'll | he's | her | here | here's | hers | herself | him | himself | his | how | how's | i | i'd | i'll | i'm | i've | if | in | into | is | isn't | it | it's | its | itself | let's | me | more | most | mustn't | my | myself | no | nor | not | of | off | on | once | only | or | other | ought | our | ours | ourselves | out | over | own | same | shan't | she | she'd | she'll | she's | should | shouldn't | so | some | such | than | that | that's | the | their | theirs | them | themselves | then | there | there's | these | they | they'd | they'll | they're | they've | this | those | through | to | tobe | too | under | until | up | very | was | wasn't | we | we'd | we'll | we'be | we're | we've | were | weren't | what | what's | when | when's | where | where's | which | while | who | who's | whom | why | why's | with | won't | would | wouldn't | you | you'd | you'll | you're | you've | your | yours | yourself | yourselves )"
simbolos = "(\\|\'|\;|\:|\?|\¿|\<|\>|\,|\*|\(|\)|\¡|\!|\$|\#|\||\%|\&|\=|\[|\])" 

In [ ]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.test_database
collection = db.test_collection
formato=".txt"
inicio=0

for cont in range(1):
    archivo = open("reuters21578/reut2-00%d%s" %(cont,".sgm"), "r")
    
    
    contenido = archivo.read()
    html = BeautifulSoup(contenido, "lxml")
    entradas = html.find_all('reuters')
  
    #Creamos un documento cn tds las noticias
    
    for i,entrada in enumerate(entradas):
        inicio=inicio+1
        texto = entrada.find('text').getText()
        documentos={"_id":inicio,
            "texto": texto}
        posts = db.posts
        post_id = posts.insert_one(documentos)
        
        
print ("TODOS LOS DOCUMENTOS HAN SIDO CREADOS CON EXITO")            



# 2.quitar stopwords

In [ ]:
#Leemos cada archivo, guardamos su informacion en la variable texto y procedemos a limpiarla
for cont in range(cant):
    texto = db.posts.find_one({'_id':cont+1}).get('texto')
    texto=texto.lower()
    texto=' '+texto+' '
    texto= texto.replace('.','')
    texto= texto.replace('\"',' ')
    texto=re.sub(simbolos,' ',texto)
    texto=re.sub('[0-9]','',texto)
    texto=re.sub(' +',' ',texto)
    texto=re.sub("( '|' )",' ',texto)
    for a in range(5):
        texto=re.sub(stopwords,' ',texto)
    texto=re.sub("'",' ',texto)    
    documentos={"_id":cont+1,
    "texto": texto}
    posts1 = db.posts1
    post1_id = posts1.insert_one(documentos)

print ("TODOS LOS DOCUMENTOS <<TEMPORALES>> HAN SIDO CREADOS CON EXITO")



# 3.Crea diccionario 

In [ ]:
lista = []
for cont in range(cant):
    data = db.posts1.find_one({'_id':cont+1}).get('texto')
    for palabra  in data.split(' '):
        if palabra not in lista:
            lista.append(palabra)
    if cont%1000==0:
        print (cont)
lista.pop(0)
print('Se han creado %d palabras en el Diccionario'%(len(lista)))

               

In [ ]:

for numero, palabra in enumerate(lista):
    numero+=1
    
    documentos={'_id':numero,
               'dato':palabra}
    diccionario = db.diccionario
    diccionario_id = diccionario.insert_one(documentos)


In [ ]:
#tf
    
for i in range(cant):
    data=db.posts1.find_one({'_id':i+1}).get('texto')
    matriz.append([])
    for j in range(len(lista)):
        contador=0
        for palabra in data.split(' '):
            if palabra== lista[j]:
                contador=contador+1
        matriz[i].append(contador)
    if i%10==0:
        print('%d filas generadas'%(i))        
tiempo_final = time() 
tiempo_ejecucion = tiempo_final - tiempo_inicial                
print ('Hecho: Tiempo de ejecución =%d' %(tiempo_ejecucion))

# indice invertido 

In [ ]:
indiceinv=[]
for j in range(len(lista)):
    contador=0
    indiceinv.append([])
    for i in range(cant):
        if matriz[i][j]!=0:
            indiceinv[j].append(i+1)
print(indiceinv)          

In [ ]:
#Total palabras
palabras=[]

for i in range(cant):
    suma=0
    for palabra in matriz[i]:
        suma+=palabra       
    palabras.append(suma)
print (palabras) 


# Bag of words

In [ ]:
matrizbowfila=[]
matrizbow=[]

for k,i in enumerate(palabras):
    matrizbowfila=[]
    for j, valor in enumerate(matriz[k]):
        
        valor=float(valor)
        i=float(i)
        almacen=valor/i
        matrizbowfila.append(almacen)
    matrizbow.append(matrizbowfila)

# Funciones para tratamiento de la query

In [ ]:
def busqueda(query):
    query=' '+query+' '
    query=query.lower()
    query= query.replace('.','')
    query= query.replace('\"',' ')
    query=re.sub(simbolos,' ',query)
    query=re.sub('[0-9]','',query)
    query=re.sub(' +',' ',query)
    query=re.sub("( '|' )",' ',query)
    for a in range(5):
        query=re.sub(stopwords,' ',query)
    return query

def diccionario_query(query):
    dicquery=query.split(" ")
    dicquery.pop(0)
    dicquery.pop(-1)
    return dicquery


def frec_query(listaquery):
    frec_query=[]*len(lista)
    for i in range(len(lista)):
        frec_query.append(0)
    for i in range(len(listaquery)):
        if listaquery[i] in lista:
            ind=lista.index(listaquery[i])
            valor = frec_query[ind]+1
            frec_query[ind]=valor
    return frec_query    
        

def bow_query(frec_query, query):
    bow_query=[]
    for i in range(len(frec_query)):
        bow_query.append(0)
    for i in range(len(frec_query)):
        if frec_query[i]!=0:
            bow_query[i]=float(frec_query[i])/float(len(query))
    return bow_query        


def consul_invertido(frec_query):
    arraydocumento=[]
    for i in range(len(frec_query)):
        if frec_query[i]!=0:
            arraydocumento=list(set().union(arraydocumento, indiceinv[i]))
    return arraydocumento        

def desempeno(arraydocumento, bow_query):
    resultados=[]
    for i in range(len(arraydocumento)):
        c=arraydocumento[i]
        suma=0
        for r in range(len(lista)):
            minimo=min(bow_query[r],matrizbow[c-1][r])
            suma=suma+minimo
        resultados.append(suma)
    return resultados
    
      



In [ ]:

button=widgets.Button(description="Buscar")
text=widgets.Text()
display(text) 
display(button)

def mititulo(documentos, resultados):
    display(HTML('<h1>RESULTADOS</h1>'))
    for i in range(len(documentos)):
        valor=max(resultados)
        pos=resultados.index(valor)
        noticia=documentos[pos]
        
        texto = db.posts.find_one({'_id':noticia}).get('texto')
        desc=islice(texto,0,100)
        descripcion=""
        for linea in desc:
            descripcion=descripcion+linea
        descripcion= descripcion.replace('\n',' ')
        
        display(HTML('<b><a href="noticias/noticia%d.txt" target="_blank">[%d] </a></b>%s...'%(noticia,i+1,descripcion)))
        resultados.pop(pos)
        documentos.pop(pos)

          
        

        
        
def handle_submit (sender):
    busqueda(text.value)
    


def actionboton(b):
    direccion=open(text.value,'r')
    archivo=direccion.read()
    direccion.close()
    query=busqueda(archivo)
    lista=diccionario_query(query)
    queryfrecuencias=frec_query(lista)
    bowquery=bow_query(queryfrecuencias, lista)
    documentos=consul_invertido(queryfrecuencias)
    producto=desempeno(documentos,bowquery)
    mititulo(documentos, producto)
    
    
    
    
text.on_submit(handle_submit)
button.on_click(actionboton)
